In [1]:
# Import required libraries
from binance.client import Client
import pandas as pd
from datetime import datetime, timedelta
import os
import time
from tqdm import tqdm

ModuleNotFoundError: No module named 'binance'

### Download OHLCV Data

In [ ]:
# This script downloads the data

def download_binance_data(symbol, market_type='futures', start_date=None, end_date=None):
    """
    Download 1-minute OHLCV data from Binance
    market_type: 'spot' or 'futures'
    """
    
    # Initialize Binance client
    client = Client()
    
    # Create data directory if it doesn't exist
    if not os.path.exists('data'):
        os.makedirs('data')
    
    # Set default dates if not provided
    if not start_date:
        start_date = (datetime.now() - timedelta(days=1)).strftime("%d %b %Y")
    if not end_date:
        end_date = datetime.now().strftime("%d %b %Y")
    
    # Convert dates to datetime objects for calculation
    start_dt = datetime.strptime(start_date, "%d %b %Y")
    end_dt = datetime.strptime(end_date, "%d %b %Y")
    
    try:
        # Calculate total days for progress bar
        total_days = (end_dt - start_dt).days + 1
        
        # Process data in daily chunks with progress bar
        with tqdm(total=total_days, desc=f"Downloading {symbol} {market_type} data") as pbar:
            all_klines = []
            current_date = start_dt
            
            while current_date <= end_dt:
                next_date = current_date + timedelta(days=1)
                
                # Get historical klines/candlestick data for each day
                if market_type.lower() == 'futures':
                    klines = client.futures_historical_klines(
                        symbol=symbol,
                        interval=Client.KLINE_INTERVAL_1MINUTE,
                        start_str=current_date.strftime("%d %b %Y"),
                        end_str=next_date.strftime("%d %b %Y")
                    )
                else:  # spot market
                    klines = client.get_historical_klines(
                        symbol=symbol,
                        interval=Client.KLINE_INTERVAL_1MINUTE,
                        start_str=current_date.strftime("%d %b %Y"),
                        end_str=next_date.strftime("%d %b %Y")
                    )
                
                all_klines.extend(klines)
                current_date = next_date
                pbar.update(1)
                time.sleep(0.1)  # Small delay to avoid rate limits
        
        # Create DataFrame
        df = pd.DataFrame(all_klines, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_volume', 'trades', 'taker_buy_base',
            'taker_buy_quote', 'ignored'
        ])
        
        # Convert timestamp to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        
        # Select and convert relevant columns to float
        df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = df[col].astype(float)
            
        # Save to CSV
        filename = f'data/{symbol}_{market_type}_{start_date.replace(" ", "_")}_{end_date.replace(" ", "_")}.csv'
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
        
    except Exception as e:
        print(f"Error downloading data: {str(e)}")